In [8]:
import tensorflow as tf
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

Dataset= pd.read_csv("Training.csv")


Dataset=Dataset.dropna() # remove null values
Dataset=Dataset.drop("Id",axis=1) # remove sample information
#print(Dataset.value_counts(Dataset["Class"]))
Labels=Dataset[["Class"]].copy() # generate separate class dataframe
Dataset=Dataset.drop("Class",axis=1) # drop class from feature dataframe
mapping = {'A': 1, 'B': 0} # change object from str to int 
Dataset['EJ'] = Dataset['EJ'].replace(mapping)
#print(Dataset.shape)
Dataset_train, Dataset_test, Labels_train, Labels_test = train_test_split(Dataset, Labels, test_size=0.5, random_state=42)
#print(type(Dataset_train))
sm = SMOTE(random_state=42)
AltDataset, AltLabels = sm.fit_resample(Dataset_train, Labels_train)
#print(AltLabels.value_counts(AltLabels["Class"]))
AltDataset = tf.convert_to_tensor(AltDataset.values, dtype=tf.float32)
AltLabels=tf.convert_to_tensor(AltLabels.values, dtype=tf.float32)
print(len(Labels_train))
DiseaseModel=keras.Sequential([layers.Dense(300,activation="relu"),layers.Dense(1,activation="sigmoid")])
DiseaseModel.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
DiseaseModel.fit(AltDataset,AltLabels,epochs=30,batch_size=150)
print("I will determine the accuracy of my test data.")
Evaluations=DiseaseModel.evaluate(Dataset_test,Labels_test)

274
Epoch 1/30
3/3 [==============================] - 1s 4ms/step - loss: 619.1911 - accuracy: 0.4910
Epoch 2/30
3/3 [==============================] - 0s 4ms/step - loss: 246.7575 - accuracy: 0.4751
Epoch 3/30
3/3 [==============================] - 0s 5ms/step - loss: 228.4127 - accuracy: 0.5226
Epoch 4/30
3/3 [==============================] - 0s 6ms/step - loss: 167.4783 - accuracy: 0.6222
Epoch 5/30
3/3 [==============================] - 0s 5ms/step - loss: 195.0534 - accuracy: 0.6290
Epoch 6/30
3/3 [==============================] - 0s 6ms/step - loss: 145.4631 - accuracy: 0.6538
Epoch 7/30
3/3 [==============================] - 0s 6ms/step - loss: 84.8205 - accuracy: 0.5747
Epoch 8/30
3/3 [==============================] - 0s 5ms/step - loss: 80.8800 - accuracy: 0.5837
Epoch 9/30
3/3 [==============================] - 0s 5ms/step - loss: 58.0942 - accuracy: 0.6425
Epoch 10/30
3/3 [==============================] - 0s 4ms/step - loss: 52.7888 - accuracy: 0.6516
Epoch 11/30
3/3 [==

In [14]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

def create_model():
    model = Sequential([Dense(300, activation="relu"), Dense(1, activation="sigmoid")])
    return model

loss_function = tf.keras.losses.BinaryCrossentropy()

num_models = 5
models = [create_model() for _ in range(num_models)]  # Initialize models

for generation in range(min(50, len(AltDataset))):  # For each generation
    losses = []
    for model in models:  # For each model
        sample, label = AltDataset[generation], AltLabels[generation]
        optimizer = tf.keras.optimizers.Adam()  # Define optimizer inside the loop
        
        # Train the model on the sample
        with tf.GradientTape() as tape:
            prediction = model(sample[None, ...], training=True)  # Forward pass
            loss = loss_function(label[None, ...], prediction)  # Compute the loss

        gradients = tape.gradient(loss, model.trainable_variables)  # Compute the gradients
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))  # Update the weights

        losses.append(loss)

    # Find the best two models
    best_two_models = [models[i] for i in np.argsort(losses)[:2]]

    # Create 5 new models for the next generation
    new_models = []
    for _ in range(num_models):
        new_model = create_model()
        new_model.build((None, sample.shape[0]))  # Build the model to create weights

        # Combine weights from the two best models
        for i in range(len(new_model.weights)):
            new_model.weights[i].assign((best_two_models[0].weights[i] + best_two_models[1].weights[i]) / 2)

        new_models.append(new_model)

    models = new_models
    


In [49]:
print(models)
from tensorflow.keras.metrics import BinaryAccuracy
for i, model in enumerate(models):
    accuracy_metric = BinaryAccuracy()
    y_pred = model.predict(Dataset_test)  # assuming x_test is your test data
    y_pred = (y_pred > 0.5).astype(int)  # convert probabilities to class labels (0 or 1)
    accuracy_metric.update_state(Labels_test.values, y_pred)  # assuming y_test is your test labels
    accuracy = accuracy_metric.result().numpy()
    print(f"Model {i + 1} accuracy: {accuracy * 100}%")
    predictions_tensor = tf.convert_to_tensor(y_pred, dtype=tf.int32)
    precision = tf.keras.metrics.Precision()
    precision.update_state(Labels_test.values, predictions_tensor)
    precision_score = precision.result().numpy()
    print(f'Model precision: {precision_score*100:.2f}%')

Proportion=(Labels_test.value_counts(Labels_test["Class"]))
print(str(Proportion[1]/(Proportion[0]+Proportion[1])*100)+"%")

[<keras.engine.sequential.Sequential object at 0x000001E986FC21F0>, <keras.engine.sequential.Sequential object at 0x000001E986FA6B50>, <keras.engine.sequential.Sequential object at 0x000001E987033460>, <keras.engine.sequential.Sequential object at 0x000001E987039910>, <keras.engine.sequential.Sequential object at 0x000001E987045D60>]
9/9 [==============================] - 0s 1ms/step
Model 1 accuracy: 81.75182342529297%
Model precision: 40.00%
9/9 [==============================] - 0s 2ms/step
Model 2 accuracy: 81.75182342529297%
Model precision: 40.00%
9/9 [==============================] - 0s 2ms/step
Model 3 accuracy: 81.75182342529297%
Model precision: 40.00%
9/9 [==============================] - 0s 2ms/step
Model 4 accuracy: 81.75182342529297%
Model precision: 40.00%
9/9 [==============================] - 0s 1ms/step
Model 5 accuracy: 81.75182342529297%
Model precision: 40.00%
17.88321167883212%


In [50]:
import tensorflow as tf

# Assuming `model` is your trained model
# Assuming `X_test` is your test data and `y_test` are your test labels
print(Labels_test.value_counts())
# Get the model's predictions for the test dataset
predictions = DiseaseModel.predict(Dataset_test)
# As model.predict returns probabilities, we need to convert them to class labels
# You can adjust the threshold according to your problem, but commonly it's 0.5 for binary classification
predictions = (predictions > 0.5).astype(int)

# Convert labels and predictions to TensorFlow tensors
predictions_tensor = tf.convert_to_tensor(predictions, dtype=tf.int32)

# Compute precision
precision = tf.keras.metrics.Precision()
precision.update_state(Labels_test.values, predictions_tensor)
precision_score = precision.result().numpy()

print(f'Model precision: {precision_score*100:.2f}%')
Calculation=(18/110*100)
Calculation="{:.2f}".format(Calculation)
print(("Through random chance the precision would be as high as {A}% ,this demonstrates that our model with a score of 38.46% precision learns some of the global patterns not specific to the training data.").format(A=Calculation))

Class
0        225
1         49
Name: count, dtype: int64
9/9 [==============================] - 0s 2ms/step
Model precision: 42.11%
Through random chance the precision would be as high as 16.36% ,this demonstrates that our model with a score of 38.46% precision learns some of the global patterns not specific to the training data.
